# Image Generation Prompt Iteration with DSPy

This notebook demonstrates how to use DSPy for iteratively improving image generation prompts using feedback and optimization.

Based on the DSPy tutorial: [Image Generation Prompt iteration](https://dspy.ai/tutorials/image_generation_prompting/)

## Setup

Import necessary libraries and configure the environment.

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv
import base64
import requests
from typing import List, Dict
from PIL import Image
import io

# Load environment variables
load_dotenv('../../.env')

## Language Model Configuration

Set up DSPy with a language model for prompt optimization.

In [ ]:
print_step("Setting up Language Model", "Configuring DSPy for prompt iteration")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=1000)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!", "Status")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")

## Image Generation Signatures

Define signatures for prompt generation and optimization.

In [ ]:
class GenerateImagePrompt(dspy.Signature):
    """Generate a detailed prompt for image generation based on requirements."""
    
    description = dspy.InputField(desc="High-level description of the desired image")
    style = dspy.InputField(desc="Artistic style or technique desired")
    quality_requirements = dspy.InputField(desc="Quality and technical requirements")
    detailed_prompt = dspy.OutputField(desc="Detailed, optimized prompt for image generation")

class AnalyzeImageQuality(dspy.Signature):
    """Analyze an image and provide feedback on quality and adherence to requirements."""
    
    original_description = dspy.InputField(desc="Original image description")
    image_analysis = dspy.InputField(desc="Description of the generated image")
    quality_score = dspy.OutputField(desc="Quality score from 1-10")
    improvement_suggestions = dspy.OutputField(desc="Specific suggestions for prompt improvement")

class RefineImagePrompt(dspy.Signature):
    """Refine an image generation prompt based on feedback."""
    
    original_prompt = dspy.InputField(desc="Original image generation prompt")
    feedback = dspy.InputField(desc="Feedback and improvement suggestions")
    refined_prompt = dspy.OutputField(desc="Improved prompt incorporating feedback")

class GenerateVariations(dspy.Signature):
    """Generate multiple prompt variations for A/B testing."""
    
    base_prompt = dspy.InputField(desc="Base prompt to create variations from")
    num_variations = dspy.InputField(desc="Number of variations to generate")
    variations = dspy.OutputField(desc="List of prompt variations with different approaches")

## Mock Image Generation Service

Since we can't actually generate images in this demo, we'll create a mock service that simulates image generation feedback.

In [ ]:
class MockImageGenerator:
    """Mock image generation service that simulates image creation and analysis."""
    
    def __init__(self):
        self.generation_history = []
    
    def generate_image(self, prompt: str) -> Dict:
        """Simulate image generation and return mock results."""
        
        # Simulate different quality based on prompt characteristics
        quality_factors = {
            "detailed": 2,
            "specific": 2, 
            "lighting": 1,
            "composition": 1,
            "style": 1,
            "high resolution": 1,
            "professional": 1,
            "artistic": 1
        }
        
        # Calculate quality score based on prompt content
        quality_score = 5  # Base score
        for factor, points in quality_factors.items():
            if factor in prompt.lower():
                quality_score += points
        
        quality_score = min(10, quality_score)  # Cap at 10
        
        # Generate mock feedback based on quality
        if quality_score >= 8:
            feedback = "Excellent quality. Image matches requirements well."
        elif quality_score >= 6:
            feedback = "Good quality. Some minor improvements possible."
        else:
            feedback = "Needs improvement. Image lacks detail or clarity."
        
        # Simulate image analysis
        image_analysis = self._analyze_prompt_for_image(prompt)
        
        result = {
            "prompt": prompt,
            "quality_score": quality_score,
            "feedback": feedback,
            "image_analysis": image_analysis,
            "success": True
        }
        
        self.generation_history.append(result)
        return result
    
    def _analyze_prompt_for_image(self, prompt: str) -> str:
        """Generate mock image analysis based on prompt content."""
        
        elements = []
        
        if "portrait" in prompt.lower():
            elements.append("human subject in focus")
        if "landscape" in prompt.lower():
            elements.append("wide scenic view")
        if "lighting" in prompt.lower():
            elements.append("dramatic lighting effects")
        if "color" in prompt.lower() or "colorful" in prompt.lower():
            elements.append("vibrant color palette")
        if "background" in prompt.lower():
            elements.append("detailed background elements")
        
        if not elements:
            elements.append("general composition")
        
        return f"Image contains: {', '.join(elements)}. Overall composition follows prompt guidelines."

# Initialize mock image generator
image_generator = MockImageGenerator()

# Test the mock generator
test_prompt = "A detailed portrait of a person with dramatic lighting and colorful background"
test_result = image_generator.generate_image(test_prompt)

print_result(f"Test generation result: {test_result}")

## Image Prompt Optimization Module

Create a module that iteratively improves image generation prompts.

In [ ]:
class ImagePromptOptimizer(dspy.Module):
    """Module for optimizing image generation prompts through iteration."""
    
    def __init__(self, max_iterations: int = 3):
        super().__init__()
        self.generate_prompt = dspy.ChainOfThought(GenerateImagePrompt)
        self.analyze_quality = dspy.ChainOfThought(AnalyzeImageQuality)
        self.refine_prompt = dspy.ChainOfThought(RefineImagePrompt)
        self.generate_variations = dspy.ChainOfThought(GenerateVariations)
        self.image_generator = MockImageGenerator()
        self.max_iterations = max_iterations
    
    def forward(self, description: str, style: str = "photorealistic", 
                quality_requirements: str = "high quality, detailed"):
        """Optimize an image prompt through iterative refinement."""
        
        print_step("Image Prompt Optimization", f"Optimizing prompt for: {description}")
        
        # Generate initial prompt
        print_step("Step 1: Initial Prompt Generation")
        initial_result = self.generate_prompt(
            description=description,
            style=style,
            quality_requirements=quality_requirements
        )
        
        current_prompt = initial_result.detailed_prompt
        print_result(f"Initial prompt: {current_prompt}", "Generated Prompt")
        
        best_prompt = current_prompt
        best_score = 0
        iteration_results = []
        
        for iteration in range(self.max_iterations):
            print_step(f"Iteration {iteration + 1}: Generate and Evaluate")
            
            # Generate image with current prompt
            generation_result = self.image_generator.generate_image(current_prompt)
            current_score = generation_result["quality_score"]
            
            print_result(f"Quality Score: {current_score}/10", "Evaluation")
            print_result(generation_result["image_analysis"], "Image Analysis")
            
            # Analyze quality and get improvement suggestions
            analysis_result = self.analyze_quality(
                original_description=description,
                image_analysis=generation_result["image_analysis"]
            )
            
            print_result(analysis_result.improvement_suggestions, "Improvement Suggestions")
            
            # Track best result
            if current_score > best_score:
                best_score = current_score
                best_prompt = current_prompt
            
            iteration_results.append({
                "iteration": iteration + 1,
                "prompt": current_prompt,
                "score": current_score,
                "feedback": generation_result["feedback"],
                "suggestions": analysis_result.improvement_suggestions
            })
            
            # If we've reached good quality or last iteration, stop
            if current_score >= 8 or iteration == self.max_iterations - 1:
                break
            
            # Refine prompt for next iteration
            print_step(f"Refining prompt for iteration {iteration + 2}")
            refinement_result = self.refine_prompt(
                original_prompt=current_prompt,
                feedback=analysis_result.improvement_suggestions
            )
            
            current_prompt = refinement_result.refined_prompt
            print_result(f"Refined prompt: {current_prompt}", "Next Iteration Prompt")
        
        return dspy.Prediction(
            best_prompt=best_prompt,
            best_score=best_score,
            final_prompt=current_prompt,
            iteration_results=iteration_results,
            total_iterations=len(iteration_results)
        )
    
    def generate_prompt_variations(self, base_prompt: str, num_variations: int = 3):
        """Generate multiple variations of a prompt for A/B testing."""
        
        print_step("Generating Prompt Variations", f"Creating {num_variations} variations")
        
        variations_result = self.generate_variations(
            base_prompt=base_prompt,
            num_variations=str(num_variations)
        )
        
        # Parse variations (in real implementation, this would be more sophisticated)
        variations_text = variations_result.variations
        
        print_result(variations_text, "Generated Variations")
        
        return variations_text

# Initialize the optimizer
prompt_optimizer = ImagePromptOptimizer(max_iterations=3)

## Example 1: Portrait Photography Optimization

Optimize a prompt for portrait photography.

In [ ]:
# Portrait photography example
portrait_description = "Professional headshot of a business executive"
portrait_style = "professional photography"
portrait_requirements = "high resolution, good lighting, sharp focus, neutral background"

result = prompt_optimizer(
    description=portrait_description,
    style=portrait_style,
    quality_requirements=portrait_requirements
)

print_step("Portrait Optimization Results")
print(f"✓ Optimization completed in {result.total_iterations} iterations")
print(f"✓ Best quality score: {result.best_score}/10")
print(f"✓ Best prompt: {result.best_prompt}")

# Show iteration progress
print_step("Iteration Progress")
for iteration_data in result.iteration_results:
    print(f"Iteration {iteration_data['iteration']}: Score {iteration_data['score']}/10")

## Example 2: Artistic Landscape Optimization

Optimize a prompt for artistic landscape generation.

In [ ]:
# Artistic landscape example
landscape_description = "Serene mountain lake at sunset with reflections"
landscape_style = "impressionist painting style"
landscape_requirements = "warm colors, soft lighting, artistic composition, detailed reflections"

landscape_result = prompt_optimizer(
    description=landscape_description,
    style=landscape_style, 
    quality_requirements=landscape_requirements
)

print_step("Landscape Optimization Results")
print(f"✓ Final prompt optimized: {landscape_result.best_prompt}")
print(f"✓ Quality improvement achieved: {landscape_result.best_score}/10")

# Generate variations for A/B testing
variations = prompt_optimizer.generate_prompt_variations(
    base_prompt=landscape_result.best_prompt,
    num_variations=3
)

## Example 3: Product Photography Optimization

Optimize prompts for e-commerce product photography.

In [ ]:
# Product photography example
product_description = "Elegant jewelry piece for online store"
product_style = "commercial product photography"
product_requirements = "clean white background, professional lighting, high detail, commercial quality"

product_result = prompt_optimizer(
    description=product_description,
    style=product_style,
    quality_requirements=product_requirements
)

print_step("Product Photography Results")
print(f"✓ Commercial-quality prompt developed")
print(f"✓ Best prompt: {product_result.best_prompt}")
print(f"✓ Achieved quality score: {product_result.best_score}/10")

## Batch Prompt Optimization

Optimize multiple prompts in parallel for different use cases.

In [ ]:
class BatchPromptOptimizer(dspy.Module):
    """Optimize multiple image prompts in batch."""
    
    def __init__(self):
        super().__init__()
        self.single_optimizer = ImagePromptOptimizer(max_iterations=2)  # Fewer iterations for batch
    
    def optimize_batch(self, prompt_requests: List[Dict]) -> List[Dict]:
        """Optimize a batch of prompt requests."""
        
        print_step("Batch Prompt Optimization", f"Processing {len(prompt_requests)} requests")
        
        results = []
        
        for i, request in enumerate(prompt_requests):
            print_step(f"Batch Item {i+1}: {request['description'][:50]}...")
            
            result = self.single_optimizer(
                description=request['description'],
                style=request.get('style', 'photorealistic'),
                quality_requirements=request.get('quality_requirements', 'high quality')
            )
            
            results.append({
                'original_request': request,
                'optimized_prompt': result.best_prompt,
                'quality_score': result.best_score,
                'iterations': result.total_iterations
            })
            
            print_result(f"Optimized for batch item {i+1}")
        
        return results

# Test batch optimization
batch_optimizer = BatchPromptOptimizer()

batch_requests = [
    {
        'description': 'Cozy coffee shop interior with warm lighting',
        'style': 'architectural photography',
        'quality_requirements': 'warm atmosphere, detailed interior, inviting mood'
    },
    {
        'description': 'Abstract geometric art with vibrant colors',
        'style': 'digital art',
        'quality_requirements': 'bold colors, clean geometry, modern aesthetic'
    },
    {
        'description': 'Wildlife photograph of a majestic eagle in flight',
        'style': 'nature photography',
        'quality_requirements': 'sharp focus, natural habitat, dynamic action'
    }
]

batch_results = batch_optimizer.optimize_batch(batch_requests)

print_step("Batch Optimization Summary")
for i, result in enumerate(batch_results):
    print(f"Item {i+1}: Score {result['quality_score']}/10 in {result['iterations']} iterations")

## Advanced Prompt Engineering Techniques

Implement advanced techniques for image prompt optimization.

In [ ]:
class AdvancedPromptTechniques(dspy.Module):
    """Advanced techniques for image prompt engineering."""
    
    def __init__(self):
        super().__init__()
        self.style_expert = dspy.ChainOfThought(
            "artistic_style, subject_matter -> style_specific_enhancements"
        )
        self.technical_expert = dspy.ChainOfThought(
            "image_type, quality_requirements -> technical_parameters"
        )
        self.composition_expert = dspy.ChainOfThought(
            "subject, mood, purpose -> composition_guidelines"
        )
    
    def enhance_with_expertise(self, base_prompt: str, enhancement_type: str) -> str:
        """Enhance a prompt using specialized expertise."""
        
        print_step(f"Applying {enhancement_type} Enhancement")
        
        if enhancement_type == "style":
            result = self.style_expert(
                artistic_style="determined from prompt",
                subject_matter=base_prompt
            )
            enhancement = result.style_specific_enhancements
            
        elif enhancement_type == "technical":
            result = self.technical_expert(
                image_type="determined from prompt",
                quality_requirements="high quality output"
            )
            enhancement = result.technical_parameters
            
        elif enhancement_type == "composition":
            result = self.composition_expert(
                subject=base_prompt,
                mood="determined from context",
                purpose="optimized visual impact"
            )
            enhancement = result.composition_guidelines
        
        else:
            enhancement = "No enhancement applied"
        
        print_result(enhancement, f"{enhancement_type.title()} Enhancement")
        
        enhanced_prompt = f"{base_prompt}. {enhancement}"
        return enhanced_prompt
    
    def apply_prompt_patterns(self, base_prompt: str) -> Dict[str, str]:
        """Apply different prompt engineering patterns."""
        
        patterns = {}
        
        # Pattern 1: Detailed Description Pattern
        patterns["detailed"] = f"Highly detailed, {base_prompt}, intricate details, masterpiece quality"
        
        # Pattern 2: Photography Pattern  
        patterns["photography"] = f"Professional photograph of {base_prompt}, shot with high-end camera, perfect lighting"
        
        # Pattern 3: Artistic Pattern
        patterns["artistic"] = f"Artistic rendering of {base_prompt}, creative interpretation, visually striking"
        
        # Pattern 4: Technical Pattern
        patterns["technical"] = f"{base_prompt}, 8K resolution, sharp focus, professional grade, studio lighting"
        
        print_step("Prompt Pattern Applications")
        for pattern_name, pattern_prompt in patterns.items():
            print_result(f"{pattern_name}: {pattern_prompt}")
        
        return patterns

# Test advanced techniques
advanced_techniques = AdvancedPromptTechniques()

base_test_prompt = "A vintage motorcycle parked in an urban alley"

# Apply different enhancements
style_enhanced = advanced_techniques.enhance_with_expertise(base_test_prompt, "style")
technical_enhanced = advanced_techniques.enhance_with_expertise(base_test_prompt, "technical")
composition_enhanced = advanced_techniques.enhance_with_expertise(base_test_prompt, "composition")

# Apply prompt patterns
pattern_variations = advanced_techniques.apply_prompt_patterns(base_test_prompt)

print_step("Advanced Enhancement Results")
print("✓ Style-enhanced prompt created")
print("✓ Technical parameters added")
print("✓ Composition guidelines applied")
print("✓ Multiple pattern variations generated")

## Prompt Quality Metrics and Evaluation

Implement comprehensive evaluation metrics for image prompts.

In [ ]:
class PromptQualityEvaluator:
    """Evaluate image prompt quality using multiple metrics."""
    
    def __init__(self):
        self.quality_factors = {
            'specificity': 0.25,    # How specific and detailed
            'clarity': 0.25,        # How clear and unambiguous  
            'completeness': 0.20,   # Includes all necessary elements
            'technical': 0.15,      # Technical parameters included
            'creativity': 0.15      # Creative and engaging elements
        }
    
    def evaluate_prompt(self, prompt: str) -> Dict[str, float]:
        """Evaluate a prompt across multiple quality dimensions."""
        
        scores = {}
        
        # Specificity score
        specific_words = ['detailed', 'high resolution', 'professional', 'specific', 'precise']
        specificity = sum(1 for word in specific_words if word in prompt.lower()) / len(specific_words)
        scores['specificity'] = min(1.0, specificity * 2)  # Scale to 0-1
        
        # Clarity score (based on sentence structure and clear descriptions)
        clarity_indicators = ['clear', 'sharp', 'defined', 'crisp', 'distinct']
        clarity = sum(1 for word in clarity_indicators if word in prompt.lower()) / len(clarity_indicators)
        scores['clarity'] = min(1.0, clarity * 2)
        
        # Completeness score (includes subject, style, technical specs)
        completeness_elements = ['subject', 'lighting', 'background', 'style', 'quality']
        completeness = 0.6  # Base score, would be more sophisticated in real implementation
        scores['completeness'] = completeness
        
        # Technical score
        technical_terms = ['resolution', 'lighting', 'focus', 'exposure', 'composition']
        technical = sum(1 for term in technical_terms if term in prompt.lower()) / len(technical_terms)
        scores['technical'] = min(1.0, technical * 2)
        
        # Creativity score
        creative_words = ['artistic', 'creative', 'unique', 'innovative', 'imaginative']
        creativity = sum(1 for word in creative_words if word in prompt.lower()) / len(creative_words)
        scores['creativity'] = min(1.0, creativity * 2)
        
        # Overall weighted score
        overall_score = sum(scores[factor] * weight for factor, weight in self.quality_factors.items())
        scores['overall'] = overall_score
        
        return scores
    
    def compare_prompts(self, prompts: List[str]) -> Dict:
        """Compare multiple prompts and rank them."""
        
        evaluations = []
        
        for i, prompt in enumerate(prompts):
            scores = self.evaluate_prompt(prompt)
            evaluations.append({
                'prompt_id': i,
                'prompt': prompt,
                'scores': scores,
                'overall_score': scores['overall']
            })
        
        # Sort by overall score
        evaluations.sort(key=lambda x: x['overall_score'], reverse=True)
        
        return {
            'rankings': evaluations,
            'best_prompt': evaluations[0]['prompt'],
            'best_score': evaluations[0]['overall_score']
        }

# Test prompt evaluation
evaluator = PromptQualityEvaluator()

test_prompts = [
    "A cat sitting on a table",
    "Professional high-resolution photograph of a majestic orange tabby cat sitting elegantly on a polished wooden table, with soft natural lighting from a window, sharp focus, detailed fur texture, warm ambient atmosphere",
    "Artistic detailed portrait of a beautiful cat with perfect lighting and composition, creative and unique perspective, masterpiece quality",
    "Cat on table, good lighting, high quality image"
]

comparison_results = evaluator.compare_prompts(test_prompts)

print_step("Prompt Quality Evaluation Results")
for i, evaluation in enumerate(comparison_results['rankings']):
    prompt_preview = evaluation['prompt'][:60] + "..." if len(evaluation['prompt']) > 60 else evaluation['prompt']
    print(f"Rank {i+1}: Score {evaluation['overall_score']:.2f} - {prompt_preview}")

print_step("Detailed Score Breakdown for Best Prompt")
best_eval = comparison_results['rankings'][0]
for factor, score in best_eval['scores'].items():
    print(f"{factor.title()}: {score:.2f}")

## Best Practices for Image Prompt Engineering

### Key Principles:

1. **Be Specific**: Include detailed descriptions of what you want
2. **Technical Parameters**: Specify resolution, lighting, focus, etc.
3. **Style Guidelines**: Clearly indicate artistic style or approach
4. **Composition**: Describe framing, perspective, and layout
5. **Quality Indicators**: Use terms like "masterpiece", "professional", "high-quality"

### Common Prompt Structure:

```
[Subject Description] + [Style/Medium] + [Technical Parameters] + [Composition] + [Quality Modifiers]
```

### Optimization Strategies:

- **Iterative Refinement**: Start simple, add complexity through iterations
- **A/B Testing**: Generate variations and compare results
- **Feedback Integration**: Use generation results to improve prompts
- **Expert Knowledge**: Apply domain-specific expertise (photography, art, etc.)
- **Quality Metrics**: Measure and optimize based on specific criteria

### Advanced Techniques:

- **Negative Prompts**: Specify what to avoid
- **Weight Adjustments**: Emphasize important elements
- **Style Transfer**: Reference specific artists or techniques  
- **Conditional Generation**: Use multiple conditions and constraints

## Conclusion

This notebook demonstrated comprehensive techniques for optimizing image generation prompts using DSPy:

- **Iterative Optimization**: Automatically improve prompts through feedback
- **Quality Evaluation**: Measure prompt effectiveness objectively
- **Batch Processing**: Handle multiple optimization tasks efficiently
- **Advanced Techniques**: Apply expert knowledge and sophisticated patterns
- **Comparative Analysis**: Evaluate and rank different prompt approaches

These methods can significantly improve the quality and consistency of AI-generated images across various applications.